In [8]:
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv
import os

# Cargar variables de entorno desde .env
load_dotenv()

def procesar_y_guardar_lego(csv_file, db_name="dbo_lego", collection_name="lego_work"):
    """
    Carga un CSV de sets de LEGO, filtra los datos según los temas seleccionados y
    los guarda en una base de datos MongoDB.
    """

    # Obtener mongo_uri desde .env
    mongo_uri = os.getenv("MONGO_URI")

    if not mongo_uri:
        raise ValueError("La variable MONGO_URI no está definida en el archivo .env")

    # Cargar CSV
    df_lego_inicial = pd.read_csv(csv_file)

    # Lista de temas elegidos
    selected_themes = [
    "Speed Champions", "Architecture", "BrickHeadz", "Star Wars", "Ideas", "Collectable Minifigures",
    "Technic", "Minecraft", "Harry Potter", "Icons", "Ninjago", "Education", "Jurassic World",
    "Duplo", "DC Comics Super Heroes", "Marvel Super Heroes", "Creator", "City", "Friends",
    "Classic", "Disney"
    ]

    # Filtrar dataset
    df_lego_work = df_lego_inicial[df_lego_inicial['Theme'].isin(selected_themes)]

    # Eliminar columnas innecesarias
    columns_to_drop = ['Own', 'Want', 'Unnamed: 49', 'Flag2', 'Flag3', 'Flag4', 'Flag5', 'Flag6', 'Flag7', 'Flag8', 'UserNotes',
                       "Variant", "ThemeGroup", "EAN", "UPC", "UKRetailPrice", 'QtyOwned', 'QtyOwnedNew', 'QtyOwnedUsed',
                       'QtyWanted', 'WantedPriority', "CARetailPrice", "DERetailPrice", "AdditionalImageCount", "InstructionsCount",
                       "USDateAdded", "USDateRemoved", "Designers", "Image", "USItemNumber", "EUItemNumber"]

    df_lego_work = df_lego_work.drop(columns=columns_to_drop, errors='ignore')

    # Conectar a MongoDB
    client = MongoClient(mongo_uri)
    db = client[db_name]
    collection = db[collection_name]

    # Convertir el DataFrame a lista de diccionarios e insertarlo en MongoDB
    data = df_lego_work.to_dict(orient="records")
    if data:
        collection.insert_many(data)
        print(f"Datos insertados en MongoDB ({len(data)} registros).")
    else:
        print("No hay datos para insertar.")

    # Cerrar conexión
    client.close()


In [9]:
#primera funcion aplicada
csv_file = "Set_Lanzanzados.csv"
procesar_y_guardar_lego(csv_file)

C:\Users\Ema\AppData\Local\Temp\ipykernel_15668\1239290459.py:22: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lego_inicial = pd.read_csv(csv_file)


Datos insertados en MongoDB (8481 registros).


In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
from pymongo import MongoClient

def clean_lego_data(df_lego):
    df_lego['Subtheme'] = df_lego['Subtheme'].fillna('Unknown')
    columns_zero = ['Pieces', 'BrickLinkSoldPriceNew', 'BrickLinkSoldPriceNewUS', 'USRetailPrice',
                    'BrickLinkSoldPriceUsed', 'Depth', 'Height', 'Width', 'Weight', 'Minifigs', 'AgeMin', 'AgeMax']
    for col in columns_zero:
        if col in df_lego.columns:
            df_lego[col] = df_lego[col].fillna(0)
    df_lego['ImageFilename'] = df_lego['ImageFilename'].fillna('Unknown')
    df_lego['LaunchDate'] = pd.to_datetime(df_lego['LaunchDate'], errors='coerce')
    df_lego['ExitDate'] = pd.to_datetime(df_lego['ExitDate'], errors='coerce')
    df_lego['Duration'] = (df_lego['ExitDate'] - df_lego['LaunchDate']).dt.days / 365.25
    theme_median_duration = df_lego.groupby('Theme')['Duration'].median()
    for theme, median_duration in theme_median_duration.items():
        mask = (df_lego['Theme'] == theme) & df_lego['ExitDate'].isna() & df_lego['LaunchDate'].notna()
        df_lego.loc[mask, 'ExitDate'] = df_lego.loc[mask, 'LaunchDate'] + pd.to_timedelta(median_duration * 365.25, unit='D')
    mask_launch = df_lego['LaunchDate'].isna() & df_lego['YearFrom'].notna()
    df_lego.loc[mask_launch, 'LaunchDate'] = pd.to_datetime(df_lego.loc[mask_launch, 'YearFrom'].astype(int).astype(str) + '-01-01')
    df_lego['LaunchYear'] = df_lego['LaunchDate'].dt.year
    df_lego['LaunchMonth'] = df_lego['LaunchDate'].dt.month
    df_lego['ExitYear'] = df_lego['ExitDate'].dt.year
    df_lego['ExitMonth'] = df_lego['ExitDate'].dt.month
    df_lego.drop(columns=['LaunchDate', 'ExitDate', 'Duration'], inplace=True)
    df_lego['Duration'] = df_lego['ExitYear'] - df_lego['LaunchYear']
    theme_avg_duration = df_lego.groupby('Theme')['Duration'].mean()
    year_avg_duration = df_lego.groupby('LaunchYear')['Duration'].mean()
    for index, row in df_lego.iterrows():
        if pd.isna(row['ExitYear']) and not pd.isna(row['LaunchYear']):
            estimated_duration = theme_avg_duration.get(row['Theme'], year_avg_duration.get(row['LaunchYear'], None))
            if pd.notna(estimated_duration):
                df_lego.at[index, 'ExitYear'] = int(row['LaunchYear'] + round(estimated_duration))
                df_lego.at[index, 'ExitMonth'] = 12
    # Elimino la columna 'Released' y 'Duration' porque no aporta información útil
    df_lego.drop(columns=['Released'], inplace=True)
    df_lego.drop(columns=['Duration'], inplace=True)
    df_lego['PackagingType'] = df_lego['PackagingType'].replace({
        '{Not specified}': 'Unknown', 'Plastic canister': 'Canister', 'Plastic box': 'Box',
        'Metal canister': 'Canister', 'Box with handle': 'Box', 'Box with backing card': 'Box',
        'None (loose parts)': 'None'})
    df_lego['Availability'] = df_lego['Availability'].replace({
        '{Not specified}': 'Unknown', 'Promotional (Airline)': 'Promotional'})
    df_lego.loc[df_lego['Theme'] == 'Creator Expert', 'Theme'] = 'Icons'
    return df_lego

def process_lego_data(df_lego):
    current_year = datetime.now().year
    df_lego['YearsSinceExit'] = (current_year - df_lego['ExitYear']).fillna(0).astype(int)
    df_lego['PriceChange'] = ((df_lego['BrickLinkSoldPriceNew'] - df_lego['USRetailPrice']) / df_lego['USRetailPrice']) * 100
    df_lego['PriceChange'] = df_lego['PriceChange'].fillna(0)
    df_lego['ResaleDemand'] = df_lego.apply(lambda row: row['BrickLinkSoldPriceNew'] / row['BrickLinkSoldPriceUsed']
                                             if row['BrickLinkSoldPriceUsed'] > 0 else 0, axis=1)
    df_lego['AppreciationTrend'] = df_lego.apply(lambda row: row['PriceChange'] / row['YearsSinceExit']
                                                 if row['YearsSinceExit'] > 0 else 0, axis=1)
    size_labels = ['Small', 'Medium', 'Large']
    df_lego['SizeCategory'] = pd.cut(df_lego['Pieces'], bins=[0, 249, 1000, float('inf')], labels=size_labels, include_lowest=True)
    exclusive_themes = ['Star Wars', 'Modular Buildings', 'Ideas', 'Creator Expert', 'Harry Potter',
                        'Marvel Super Heroes', 'Ghostbusters', 'Icons', 'The Lord of the Rings',
                        'Pirates of the Caribbean', 'Pirates', 'Trains', 'Architecture']
    df_lego['Exclusivity'] = df_lego['Theme'].apply(lambda x: 'Exclusive' if x in exclusive_themes else 'Regular')
    theme_popularity = df_lego.groupby('Theme')['PriceChange'].mean().replace([np.inf, -np.inf], np.nan)
    df_lego['ThemePopularity'] = df_lego['Theme'].map(theme_popularity).fillna(0)
    df_lego['InvestmentScore'] = df_lego.apply(lambda row: (row['PriceChange'] * 0.4) +
                                                         (row['AppreciationTrend'] * 0.3) +
                                                         (row['ThemePopularity'] * 0.2) +
                                                         (10 if row['Exclusivity'] == 'Exclusive' else 0), axis=1)

    # Calculamos el incremento de precio anual desde que el set fue retirado
    df_lego['AnnualPriceIncrease'] = (df_lego['BrickLinkSoldPriceNew'] - df_lego['USRetailPrice']) / df_lego['YearsSinceExit']

    # Reemplazamos valores infinitos o NaN (por si hay sets con YearsSinceExit = 0)
    df_lego.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_lego['AnnualPriceIncrease'].fillna(0, inplace=True)

    # Calculamos el porcentaje de aumento anual del precio desde que el set fue retirado
    df_lego['AnnualPercentageIncrease'] = ((df_lego['BrickLinkSoldPriceNew'] - df_lego['USRetailPrice']) /
                                       (df_lego['USRetailPrice'] * df_lego['YearsSinceExit'])) * 100

    # Reemplazamos valores infinitos o NaN (por si hay YearsSinceExit o USRetailPrice en 0)
    df_lego.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_lego['AnnualPercentageIncrease'].fillna(0, inplace=True)



    return df_lego

def main():

     # Obtener mongo_uri desde .env
    mongo_uri = os.getenv("MONGO_URI")

    # Conectar a MongoDB
    client = MongoClient(mongo_uri)
    db = client['dbo_lego']  # Reemplaza con el nombre de tu BBDD
    collection = db['lego_work']
    data = pd.DataFrame(list(collection.find()))
    if '_id' in data.columns:
        data.drop(columns=['_id'], inplace=True)
    data_cleaned = clean_lego_data(data)
    data_processed = process_lego_data(data_cleaned)
    db['lego_work_final'].insert_many(data_processed.to_dict(orient='records'))
    print("Datos limpiados y guardados en lego_work_final")

if __name__ == "__main__":
    main()

C:\Users\Ema\AppData\Local\Temp\ipykernel_15668\362846168.py:15: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_lego['ExitDate'] = pd.to_datetime(df_lego['ExitDate'], errors='coerce')
C:\Users\Ema\AppData\Local\Temp\ipykernel_15668\362846168.py:76: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_lego['AnnualPriceIncrease'].fillna(0, inplace=True)
C:\Users\Ema\AppData\Local\Temp\ipykernel_15668\362846

Datos limpiados y guardados en lego_work_final


In [11]:
import os
import pandas as pd
from pymongo import MongoClient

def actualizar_lego_en_mongo():
    # Leer la URI de MongoDB desde la variable de entorno
    mongo_uri = os.getenv("MONGO_URI")
    client = MongoClient(mongo_uri)
    db = client["dbo_lego"]  # Reemplaza con el nombre correcto de tu BBDD

    # Nombre de la colección original y las de salida
    collection_original = "lego_work_final"
    collection_retirados = "lego_final_retirados"
    collection_venta = "lego_final_venta"

    # Leer datos desde MongoDB
    data = list(db[collection_original].find())
    df_lego_final = pd.DataFrame(data)

    # Verificar que la columna ExitYear existe y convertir a numérico
    df_lego_final['ExitYear'] = pd.to_numeric(df_lego_final.get('ExitYear'), errors='coerce')

    # Filtrar los datos según ExitYear
    df_lego_final_retirados = df_lego_final[df_lego_final['ExitYear'] < 2025].copy()
    df_lego_final_venta = df_lego_final[df_lego_final['ExitYear'] >= 2025].copy()

    # Convertir DataFrames a diccionarios para MongoDB
    data_retirados = df_lego_final_retirados.to_dict(orient='records')
    data_venta = df_lego_final_venta.to_dict(orient='records')

    # Eliminar colecciones si existen y crear nuevas con los datos actualizados
    db[collection_retirados].drop()
    db[collection_venta].drop()

    db[collection_retirados].insert_many(data_retirados) if data_retirados else None
    db[collection_venta].insert_many(data_venta) if data_venta else None

    print("Datos actualizados en MongoDB.")

# Ejecutar la función si el script se ejecuta directamente
if __name__ == "__main__":
    actualizar_lego_en_mongo()


Datos actualizados en MongoDB.


APLICACION ML EN VENTAS, GUARDAR EN MONGODB

In [14]:
import pandas as pd
import numpy as np
import joblib
import os
from pymongo import MongoClient, UpdateOne
from dotenv import load_dotenv
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Cargar variables de entorno
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")
MONGO_DB = os.getenv("MONGO_DB")

# Conexión a MongoDB
client = MongoClient(MONGO_URI)
db = client[MONGO_DB]
collection_venta = db["lego_final_venta"]
collection_retirados = db["lego_final_retirados"]

# Cargar datos desde MongoDB
df_lego_venta = pd.DataFrame(list(collection_venta.find()))
df_lego_retirados = pd.DataFrame(list(collection_retirados.find()))

# Eliminar el campo '_id' si existe
#df_lego_venta.drop(columns=['_id'], errors='ignore', inplace=True)
#df_lego_retirados.drop(columns=['_id'], errors='ignore', inplace=True)

# Eliminamos los sets promocionales con precio 0
df_lego_retirados = df_lego_retirados[df_lego_retirados['USRetailPrice'] > 0]
df_lego_venta = df_lego_venta[df_lego_venta['USRetailPrice'] > 0]

# Codificamos la columna Exclusivity
exclusivity_mapping = {'Regular': 0, 'Exclusive': 1}
df_lego_retirados['Exclusivity'] = df_lego_retirados['Exclusivity'].map(exclusivity_mapping)
df_lego_venta['Exclusivity'] = df_lego_venta['Exclusivity'].map(exclusivity_mapping)

# Codificamos la columna SizeCategory
size_category_mapping = {'Small': 0, 'Medium': 1, 'Large': 2}
df_lego_retirados['SizeCategory'] = df_lego_retirados['SizeCategory'].map(size_category_mapping)
df_lego_venta['SizeCategory'] = df_lego_venta['SizeCategory'].map(size_category_mapping)

# Feature Engineering
df_lego_retirados["PricePerPiece"] = df_lego_retirados["USRetailPrice"] / df_lego_retirados["Pieces"]
df_lego_venta["PricePerPiece"] = df_lego_venta["USRetailPrice"] / df_lego_venta["Pieces"]

df_lego_retirados["PricePerMinifig"] = np.where(df_lego_retirados["Minifigs"] > 0, df_lego_retirados["USRetailPrice"] / df_lego_retirados["Minifigs"], 0)
df_lego_venta["PricePerMinifig"] = np.where(df_lego_venta["Minifigs"] > 0, df_lego_venta["USRetailPrice"] / df_lego_venta["Minifigs"], 0)

df_lego_retirados["YearsOnMarket"] = df_lego_retirados["ExitYear"] - df_lego_retirados["LaunchYear"]
df_lego_venta["YearsOnMarket"] = df_lego_venta["ExitYear"] - df_lego_venta["LaunchYear"]

# Reemplazamos valores infinitos por NaN y rellenamos con la mediana en columnas numéricas
for df in [df_lego_retirados, df_lego_venta]:
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Definimos variables
features = ['USRetailPrice', 'Pieces', 'Minifigs', 'YearsSinceExit', 'ResaleDemand',
            'AnnualPriceIncrease', 'Exclusivity', 'SizeCategory', 'PricePerPiece', 'PricePerMinifig', 'YearsOnMarket']
target = 'InvestmentScore'

# Dividimos los datos en entrenamiento y prueba
X = df_lego_retirados[features]
y = df_lego_retirados[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definimos los hiperparámetros para GridSearch
rf_params = {'n_estimators': [50, 100, 150], 'max_depth': [10, 20, None], 'min_samples_split': [2, 5, 10]}
hgb_params = {'learning_rate': [0.01, 0.1, 0.2], 'max_iter': [100, 200, 300], 'max_depth': [10, 20, None]}

# GridSearch para Random Forest
rf_grid = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=5, n_jobs=-1, verbose=1)
rf_grid.fit(X_train, y_train)

# GridSearch para HistGradientBoosting
hgb_grid = GridSearchCV(HistGradientBoostingRegressor(random_state=42), hgb_params, cv=5, n_jobs=-1, verbose=1)
hgb_grid.fit(X_train, y_train)

# Obtenemos mejores hiperparámetros
best_rf_params = rf_grid.best_params_
best_hgb_params = hgb_grid.best_params_

# Definimos los modelos optimizados
rf_best = RandomForestRegressor(**best_rf_params, random_state=42)
hgb_best = HistGradientBoostingRegressor(**best_hgb_params, random_state=42)
lr_model = LinearRegression()

# Definimos modelo de Stacking
base_models_optimized = [('rf', rf_best), ('hgb', hgb_best), ('lr', lr_model)]
stacking_model_optimized = StackingRegressor(estimators=base_models_optimized, final_estimator=LinearRegression())

# Validación cruzada
cv_scores = cross_val_score(stacking_model_optimized, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)

# Entrenamos el modelo optimizado
stacking_model_optimized.fit(X_train, y_train)

# Aplicamos el modelo a los sets en venta
df_lego_venta["PredictedInvestmentScore"] = stacking_model_optimized.predict(df_lego_venta[features])

# Ordenamos por mayor potencial de inversión
#df_lego_venta_sorted = df_lego_venta.sort_values(by="PredictedInvestmentScore", ascending=False)

# ===========================
# **Actualizar MongoDB con nuevas columnas en ambas colecciones**
# ===========================

# Definir las columnas que queremos eliminar antes de actualizar
columns_to_remove = ["PricePerPiece", "PricePerMinifig", "YearsOnMarket", "PredictedInvestmentScore"]

# Paso 1: Eliminar las columnas antiguas en ambas colecciones si existen
collection_venta.update_many({}, {"$unset": {col: "" for col in columns_to_remove}})
collection_retirados.update_many({}, {"$unset": {col: "" for col in columns_to_remove}})

# Función para agregar nuevas columnas si no existen y luego actualizar los datos
def actualizar_mongodb(df, collection):
    # Verificamos si la columna existe en la colección
    existing_fields = collection.find_one()
    fields_to_add = {col: None for col in columns_to_remove if col not in existing_fields}

    # Si hay columnas que no existen, agregarlas
    if fields_to_add:
        collection.update_many({}, {"$set": fields_to_add})
        print(f"Columnas nuevas agregadas a la colección: {collection.name}")

    # Ahora, actualizamos los datos con los valores calculados
    updates = []
    for _, row in df.iterrows():
        filter_query = {"_id": row["_id"]}  # Asegurar que actualizamos el documento correcto
        update_query = {
            "$set": {
                "PricePerPiece": row["PricePerPiece"],
                "PricePerMinifig": row["PricePerMinifig"],
                "YearsOnMarket": row["YearsOnMarket"],
                "PredictedInvestmentScore": row.get("PredictedInvestmentScore", None)  # Si no existe, pone None
            }
        }
        updates.append(UpdateOne(filter_query, update_query))

    # Ejecutar las actualizaciones en MongoDB
    if updates:
        collection.bulk_write(updates)
    print(f"Datos actualizados correctamente en la colección: {collection.name}")

# Aplicamos la actualización a ambas colecciones
actualizar_mongodb(df_lego_venta, collection_venta)
actualizar_mongodb(df_lego_retirados, collection_retirados)

# Verificar si las actualizaciones se guardaron en MongoDB
doc_venta = collection_venta.find_one({}, {"PricePerPiece": 1, "PricePerMinifig": 1, "YearsOnMarket": 1, "PredictedInvestmentScore": 1})
doc_retirados = collection_retirados.find_one({}, {"PricePerPiece": 1, "PricePerMinifig": 1, "YearsOnMarket": 1, "PredictedInvestmentScore": 1})

if doc_venta:
    print("Ejemplo de documento actualizado en lego_final_venta:", doc_venta)
else:
    print("No se encontraron documentos con las nuevas columnas en lego_final_venta.")

if doc_retirados:
    print("Ejemplo de documento actualizado en lego_final_retirados:", doc_retirados)
else:
    print("No se encontraron documentos con las nuevas columnas en lego_final_retirados.")

print("Proceso completado con éxito. 🚀")


# Guardar el modelo entrenado
joblib.dump(stacking_model_optimized, "../05_Streamlit/models/stacking_model.pkl", protocol=4, compress=3)

print("\nValidación Cruzada (5-fold):")
print(f"R² Medio: {cv_scores.mean():.4f}, Desviación Estándar: {cv_scores.std():.4f}")


C:\Users\Ema\AppData\Local\Temp\ipykernel_15668\4111964515.py:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace([np.inf, -np.inf], np.nan, inplace=True)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Columnas nuevas agregadas a la colección: lego_final_venta
Datos actualizados correctamente en la colección: lego_final_venta
Columnas nuevas agregadas a la colección: lego_final_retirados
Datos actualizados correctamente en la colección: lego_final_retirados
Ejemplo de documento actualizado en lego_final_venta: {'_id': ObjectId('67c8a69e30a1f61f23a242ab'), 'PredictedInvestmentScore': 89.63012649953507, 'PricePerMinifig': 0.0, 'PricePerPiece': 0.07935185185185185, 'YearsOnMarket': 5.0}
Ejemplo de documento actualizado en lego_final_retirados: {'_id': ObjectId('67c8a69e30a1f61f23a23954'), 'PredictedInvestmentScore': None, 'PricePerMinifig': None, 'PricePerPiece': None, 'YearsOnMarket': None}
Proceso completado con éxito. 🚀

Validación Cruzada (5-fold):
R² Medio: 0.9862, Desviación Estándar: 0.0020
